<a href="https://colab.research.google.com/github/fahimabrar/Natural-Language-Processing/blob/main/Word%2520Embeddings/Embeddings_with_Keras_Embeddding_Layer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds

from tensorflow import keras 
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

import numpy as np

In [2]:
print(tf.__version__)

2.6.0


In [3]:
data, info = tfds.load("imdb_reviews", with_info = True, as_supervised= True)

In [4]:
train_data, test_data = data['train'], data['test']

In [5]:
train_sentences = []
test_sentences = []

train_labels = []
test_labels = []

In [6]:
for sent, label in train_data:
  train_sentences.append(str(sent.numpy().decode('utf8')))
  train_labels.append(label.numpy())



for sent, label in test_data:
  test_sentences.append(str(sent.numpy().decode('utf8')))
  test_labels.append(label.numpy())

In [7]:
train_sentences[0]

"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it."

In [8]:
train_labels[0]

0

In [9]:
train_labels = np.array(train_labels)
test_labels = np.array(test_labels)

In [10]:
type(train_sentences)

list

In [11]:
vocab_size = 10000
embedding_dim = 50
max_length = 150
trunc_type = "post"
oov_tok = "<oov>"

In [12]:
tokenizer = Tokenizer(num_words= vocab_size, oov_token= oov_tok)

tokenizer.fit_on_texts(train_sentences)
word_index = tokenizer.word_index
len(word_index)

88583

In [13]:
text = ""
text = " ".join(train_sentences)

In [14]:
from collections import Counter
def word_count(text):
  return Counter(text.split())

counter  = word_count(text)

In [15]:
vocab_dictionary =  dict(counter.most_common())

In [16]:
#uncomment the follwing line to see the word frequency
# vocab_dictionary


In [17]:
train_seqs = tokenizer.texts_to_sequences(train_sentences)
train_padded = pad_sequences(train_seqs, maxlen =  max_length, truncating= trunc_type)
test_seqs = tokenizer.texts_to_sequences(test_sentences)
test_padded = pad_sequences(test_seqs, maxlen =  max_length, truncating= trunc_type)

In [18]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])


model.compile(loss= 'binary_crossentropy', optimizer= 'adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 150, 50)           500000    
_________________________________________________________________
flatten (Flatten)            (None, 7500)              0         
_________________________________________________________________
dense (Dense)                (None, 6)                 45006     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 7         
Total params: 545,013
Trainable params: 545,013
Non-trainable params: 0
_________________________________________________________________


In [20]:
num_epoch = 10
model.fit(train_padded, train_labels, epochs = num_epoch, validation_data=(test_padded, test_labels))

Epoch 1/10
782/782 [==============================] - 6s 7ms/step - loss: 0.0236 - accuracy: 0.9961 - val_loss: 0.5042 - val_accuracy: 0.8273
Epoch 2/10
782/782 [==============================] - 6s 8ms/step - loss: 0.0033 - accuracy: 0.9999 - val_loss: 0.5631 - val_accuracy: 0.8286
Epoch 3/10
782/782 [==============================] - 6s 7ms/step - loss: 0.0010 - accuracy: 1.0000 - val_loss: 0.6009 - val_accuracy: 0.8318
Epoch 4/10
782/782 [==============================] - 6s 8ms/step - loss: 5.0749e-04 - accuracy: 1.0000 - val_loss: 0.6344 - val_accuracy: 0.8320
Epoch 5/10
782/782 [==============================] - 6s 7ms/step - loss: 2.8498e-04 - accuracy: 1.0000 - val_loss: 0.6671 - val_accuracy: 0.8325
Epoch 6/10
782/782 [==============================] - 6s 8ms/step - loss: 1.6482e-04 - accuracy: 1.0000 - val_loss: 0.6989 - val_accuracy: 0.8321
Epoch 7/10
782/782 [==============================] - 7s 10ms/step - loss: 9.9610e-05 - accuracy: 1.0000 - val_loss: 0.7300 - val_accura

In [21]:
l1 = model.layers[0]

In [22]:
l1

In [23]:
weights = l1.get_weights()[0]
print(weights.shape)
print(weights)

(10000, 50)
[[-0.02459711 -0.0377754  -0.02033278 ... -0.02471545  0.01482258
   0.00475845]
 [-0.04864111 -0.03107286  0.00214711 ... -0.01085368 -0.00985176
   0.02486526]
 [ 0.01042906 -0.00317019  0.00701924 ... -0.00587262 -0.01009039
   0.05139241]
 ...
 [-0.00577927 -0.04583653  0.06259663 ... -0.05582897  0.09835495
   0.06188731]
 [-0.00279514 -0.10847239  0.07720436 ... -0.07776046  0.03525668
   0.05743249]
 [ 0.04638394 -0.09902145  0.02673982 ... -0.08700522 -0.0023003
   0.00439921]]


In [25]:
weights.shape

(10000, 50)